In [1]:
import torch
import torch.nn as nn
# from functorch import make_functional, grad
from torch.func import functional_call, grad, vmap, vjp, jacrev

In [2]:
model = nn.Sequential(
    nn.Linear(3, 10),
    nn.ReLU(),
    nn.Linear(10, 20),
    nn.ReLU(),
    nn.Sequential(
        nn.Linear(20, 10),
        nn.ReLU(),
        nn.Linear(10, 20),
        nn.ReLU(),
    ),
    nn.Linear(20, 2),
    nn.Softmax(dim=-1),
)

In [3]:
# func, params = make_functional(model)
functional_call(model, dict(model.named_parameters()), torch.randn(10, 3))


tensor([[0.4415, 0.5585],
        [0.4435, 0.5565],
        [0.4496, 0.5504],
        [0.4564, 0.5436],
        [0.4343, 0.5657],
        [0.4360, 0.5640],
        [0.4431, 0.5569],
        [0.4652, 0.5348],
        [0.4528, 0.5472],
        [0.4502, 0.5498]], grad_fn=<SoftmaxBackward0>)

In [6]:
result = jacrev(lambda params, input: functional_call(model, params, input), argnums=0)(dict(model.named_parameters()), torch.randn(117, 3))

In [5]:
dict_param = dict(model.named_parameters())
for key in dict_param.keys():
    print(key, dict_param[key].shape, result[key].shape)

0.weight torch.Size([10, 3]) torch.Size([117, 2, 10, 3])
0.bias torch.Size([10]) torch.Size([117, 2, 10])
2.weight torch.Size([20, 10]) torch.Size([117, 2, 20, 10])
2.bias torch.Size([20]) torch.Size([117, 2, 20])
4.0.weight torch.Size([10, 20]) torch.Size([117, 2, 10, 20])
4.0.bias torch.Size([10]) torch.Size([117, 2, 10])
4.2.weight torch.Size([20, 10]) torch.Size([117, 2, 20, 10])
4.2.bias torch.Size([20]) torch.Size([117, 2, 20])
5.weight torch.Size([2, 20]) torch.Size([117, 2, 2, 20])
5.bias torch.Size([2]) torch.Size([117, 2, 2])


In [8]:
import numpy as np

In [12]:
torch.randn(5,4,3) * np.ones((5,4,3))

tensor([[[ 1.0450,  2.6428, -0.5227],
         [ 0.3608,  1.1736,  0.5707],
         [-0.1223, -0.3912,  0.6274],
         [ 0.0113, -0.0442, -0.9692]],

        [[-1.0180, -1.2155,  0.8685],
         [-0.2046, -0.9934, -0.7542],
         [ 1.2549,  0.0188, -0.4634],
         [-0.2680,  0.7302,  1.9944]],

        [[ 0.0304,  0.8716,  0.0114],
         [-0.8623, -1.1859, -0.0055],
         [-0.2921, -0.5316,  1.0583],
         [ 0.9751, -0.6307,  0.3232]],

        [[ 0.4677, -0.1208,  1.5119],
         [-1.4177, -1.2684,  0.0795],
         [ 0.0352,  0.3082, -0.5326],
         [ 0.7714, -0.8671,  0.0601]],

        [[-0.8817,  0.4520, -1.0624],
         [-0.9823, -0.1784,  1.8584],
         [-0.0661, -1.5817,  0.2411],
         [-2.2205, -0.6710, -1.9523]]], dtype=torch.float64)